In [1]:
import pandas as pd 
import numpy as np
import ast
import re
from bs4 import BeautifulSoup
import PyPDF2 # type: ignore
from sentence_transformers import SentenceTransformer, util # type: ignore

In [3]:
model = SentenceTransformer('msmarco-distilbert-base-tas-b')

#### Tags Text Cleaning

In [4]:
def clean_categories(categories):
    tags = [tag.strip() for tag in ast.literal_eval(categories)]
    return tags

In [5]:
df = pd.read_csv(r'arxiv-paper-abstracts\arxiv_data_210930-054931.csv')
categories = pd.DataFrame(list(df['terms'].unique()),columns=['categories'])
categories['clean_categories'] = categories['categories'].apply(clean_categories)
tags = set()
for index, row in categories.iterrows():
    categories = row['clean_categories']
    for category in categories:
        tags.add(category)
tags_df = pd.DataFrame(list(tags), columns=['tags'])
tags = set()
for index, row in tags_df.iterrows():
    categories_str = row['tags']
    categories_list = [category.strip() for category in categories_str.replace(',', ';').split(';')]
    for category in categories_list:
        tags.add(category)
tags_df = pd.DataFrame(list(tags), columns=['tags'])
tags = set()
for index, row in tags_df.iterrows():
    categories_str = row['tags']
    categories_list = [category.strip() for category in categories_str.replace(',', ';').split(';')]
    for category in categories_list:
        if '(' in category and ')' in category:
            subcategories = [subcat.strip() for subcat in category.split() if subcat[0].isdigit()]
            for subcategory in subcategories:
                tags.add(subcategory)
        else:
            tags.add(category)
tags.discard('')


In [6]:
tags_list = []
for tag in tags:
    tag = tag.replace('(Primary)', '')
    tag = tag.replace('Primary', '')
    tag = tag.replace('(Secondary)', '')
    tag = tag.replace('(secondary)', '')
    tag = tag.replace('secondary', '')
    tag = tag.replace('Secondary', '')
    tag = tag.replace('and ', '')
    tags_list.append(tag.strip())
print(tags_list)

['34C20', '94A17', '11Y16', 'I.4.m I.2.7', '91A06', '62P15', 'K.5.2', '68U20', '68N20', '68W10', '60', '91-10', 'I.1.4', 'cond-mat.soft', '62G20', '68Tx', '92C50', '37M25', '14M15', '05C80', '62R01', '62D05', '49L20', 'cs.GR', 'F.4.1', '90B22', 'I.6.6', 'astro-ph.SR', '49', '49N10', '65D17', '51M20', 'G.2.2', 'D.3.4', 'J.7', '53A17', '42-08', '58J35', '68Q87', 'E.4', '90C20', '62-08', 'cs.MM', 'cs.OS', '00Bxx', 'F.3.1', '65F22', '60G99', '2020: 49N45', 'F.1.3', 'physics.class-ph', 'J.2.', 'K.4.2', 'I.2.0', '97R40', 'A.1', '05A16', '60H30', '68P99', '60G55', 'J.5', 'eess.AS', '62E10', '60D05', '42B35', '15A29', '65C50', 'math.AT', '60H99', '05B45', 'H.2.7', '68W50', 'G.1.3', '41A10', 'physics.comp-ph', '68R10', 'I.4.4', '68-06', 'J.1', '35K57', 'G.1.0', 'B.7.2', '68T42', '47G20', '94A34', '62G10', '91D30', '62E17', 'cs.MA', 'D.2.5', '86-08', '62P35', '90C59', 'I.2.4', '68Q55', 'cs.AI', '60J20', 'q-bio.QM', 'G.3.7', '65K05', 'cs.CL', 'physics.acc-ph', '14J26', 'cs.ET', 'K.4', '57Z25', 'G

In [7]:
# create empty lists for each classification
msc = set()
ccs = set()
arxiv = set()
misc = set()

for tag in tags_list:
    if re.match(r'\d{2}[A-Z]\d{2}|\d{2}[A-Z]-xx|\d{2}\w{2,}', tag):
        msc.add(tag)
    elif re.match(r'\d{2}-\d{2}|\d{2}-xx|\d{2}-XX', tag):
        msc.add(tag)
    elif re.match(r'\d{2}', tag):
        msc.add(tag)
    elif re.match(r'[A-Z]\.\d\.\d', tag):
        ccs.add(tag)
    elif re.match(r'[A-Z]\.\d', tag):
        ccs.add(tag)
    elif re.match(r'[A-Z]\d\.\d', tag):
        ccs.add(tag)
    elif re.match(r'\w+\.\w+', tag):
        arxiv.add(tag)
    elif re.match(r'\w+-\w+\.\w+-\w+', tag):
        arxiv.add(tag)
    elif re.match(r'\w+-\w+\.\w+', tag):
        arxiv.add(tag)
    elif re.match(r'\w+-\w+', tag):
        arxiv.add(tag)
    else:
        misc.add(tag)

In [8]:
print(f'MSC: {len(msc)}\nCCS: {len(ccs)}\nArXiv: {len(arxiv)}\nMisc: {len(misc)}')

MSC: 453
CCS: 162
ArXiv: 149
Misc: 5


**Contents of misc:**
- Artificial intelligence': corresponds to 68Txx
- Computer science applications- 97R60 Computer graphics': corresponds to 97R60
- Computing methodologies for image processing': corresponds to 68U10
- This paper tells us how human can be identified by their Gait cycle\n  using any simple camera': dropped
- org: dropped

In [9]:
msc.add('97R60')
del misc

In [10]:
msc_cleaned = pd.DataFrame(list(msc), columns=['msc'])
ccs_cleaned = pd.DataFrame(list(ccs), columns=['ccs'])
arxiv_cleaned = pd.DataFrame(list(arxiv), columns=['arxiv'])

#### Retrieve Descriptions for Each Tag

In [11]:
msc_html = r'C:\Users\User\Desktop\ml_paper_recommender_system\paper-recommender\Classification Codes Data\MSC Classification Codes.html'

# Parse MSC Classification Codes html file
with open(msc_html) as fp:
    msc_soup = BeautifulSoup(fp, 'html.parser')

# Extract MSC Classification Codes
msc_tree = [element.text for element in msc_soup.find_all('li')]

# Define Dictionary for MSC Classification Codes where the key is the main class and the value is a list of sub-classes
msc_classes = {}

# Populate the dictionary
for cls in msc_tree:
    if re.match(r'^\d\d-xx:', cls):
        msc_classes[cls.split('\n')[0]] = []

for element in msc_tree:
    for cls in msc_classes.keys():
        if cls[0:2] == element[0:2]:
            msc_classes[cls].append(element)

for cls in msc_classes.keys():
    msc_classes[cls].pop(0)

# Create a dataframe from the dictionary
msc_df = pd.DataFrame(msc_classes.items(),columns=['class','sub_classes'])

In [13]:
cls = []
cls_str = []
sub_cls = []
sub_cls_str = []
sub_sub_cls = []
sub_sub_cls_str = []
for idx, sub_class in enumerate(msc_df['sub_classes'].values):
    for sub_sub in sub_class:
        if re.match(r'\d\d-\d\d', sub_sub):
            cls.append(sub_sub.split(':')[0])
            cls_str.append(sub_sub.split(':')[1].strip())
        if re.match(r'\d\d\w\w\w', sub_sub):
            sub_cls.append(sub_sub.split(':')[0])
            sub_cls_str.append(sub_sub.split(':')[1].strip())
        if re.match(r'\d\d\w\d\d', sub_sub):
            sub_sub_cls.append(sub_sub.split(':')[0])
            sub_sub_cls_str.append(sub_sub.split(':')[1].strip())
print(f'Classes: {len(cls)} {len(cls_str)}, Sub-Classes: {len(sub_cls)} {len(sub_cls_str)}, Sub-Sub-Classes: {len(sub_sub_cls)} {len(sub_sub_cls_str)}')

cls_comb = zip(cls, cls_str)
cls_comb = list(cls_comb)
sub_cls_comb = zip(sub_cls, sub_cls_str)
sub_cls_comb = list(sub_cls_comb)
sub_sub_cls_comb = zip(sub_sub_cls, sub_sub_cls_str)
sub_sub_cls_comb = list(sub_sub_cls_comb)
codes_comb = cls_comb + sub_cls_comb + sub_sub_cls_comb
codes_comb = list(codes_comb)
msc_index = pd.DataFrame(codes_comb, columns=['code', 'description'])

Classes: 397 397, Sub-Classes: 5071 5071, Sub-Sub-Classes: 4647 4647


In [14]:
msc_index

,code,description
0,00-01,"Instructional exposition (textbooks, tutorial ..."
1,00-02,"Research exposition (monographs, survey articles)"
2,01-00,"General reference works (handbooks, dictionari..."
3,01-01,"Instructional exposition (textbooks, tutorial ..."
4,01-02,"Research exposition (monographs, survey articles)"
...,...,...
10110,97U50,Computer assisted instruction and programmed i...
10111,97U60,Manipulative materials and their use in the cl...
10112,97U70,"Technological tools (computers, calculators, s..."
10113,97U80,Audiovisual media and their use in instruction


In [15]:
msc_cleaned

,msc
0,34C20
1,11Y16
2,94A17
3,91A06
4,62P15
...,...
449,93E20
450,30C40
451,03B70
452,35K08


In [45]:
dataset_codes = list(msc_cleaned.msc.values)
missing_codes = []
for code in dataset_codes:
    if code not in list(msc_index.code.values):
        missing_codes.append(code)

for idx, code in enumerate(missing_codes):
    missing_codes[idx] = missing_codes[idx].split(' ')
new_list = []
for code in missing_codes:
    for element in code:
        if not isinstance(element, list):
            new_list.append([element])
missing_codes = [item for sublist in new_list for item in sublist]
for idx, code in enumerate(missing_codes):
    if code not in list(msc_index.code.values):
        missing_codes.remove(code)
print(f'Remaining Missing Codes: {len(missing_codes)}')

Remaining Missing Codes: 46


In [47]:
msc_cleaned['description'] = None

In [55]:
correct = []
incorrect = []
for code in missing_codes:
    if re.match(r'\d\d-\d\d', code):
        correct.append(code)
    elif re.match(r'\d\d\w\w\w', code):
        correct.append(code)
    elif re.match(r'\d\d\w\d\d', code):
        correct.append(code)
    elif re.match(r'\d\d-\w\w', code):
        correct.append(code)
    else:
        incorrect.append(code)

print(incorrect)

['60', '6006', '93', '68', '68:', 'science', '62', '6804', '00', '2010']


In [56]:
# Extract Description from MSC Classification Codes 2020
reader = PyPDF2.PdfReader(r'C:\Users\User\Desktop\ml_paper_recommender_system\paper-recommender\Classification Codes Data\msc2020.pdf')
missing_corpus = []
for idx, page in enumerate(reader.pages):
    text = page.extract_text()
    for idx, code in enumerate(correct):
        if code in text:
            missing_corpus.append(text)
            # print(f'Page {idx+1}: {code}')

In [58]:
for idx, text in enumerate(missing_corpus):
    missing_corpus[idx] = text.split('\n')

In [59]:
missing_descriptions = set()
for code in missing_codes:
    for idx, text in enumerate(missing_corpus):
        for line in text:
            if code in line:
                missing_descriptions.add(line)

In [60]:
missing_descriptions

{'(aspects in computer science) [See also 94Axx, 94Bxx]',
 '05B05 Combinatorial aspects of block designs [See also 51E05, 62K10]',
 '05C21 Flows in graphs',
 '05C60 Isomorphism problems in graph theory (reconstruction conjecture, etc.) and homomorphisms (subgraph',
 '05C62 Graph representations (geometric and intersection representations, etc.) fFor graph drawing, see also 68R10 g',
 '05C80 Random graphs (graph-theoretic aspects) [See also 60B20]',
 '05C81 Random walks on graphs',
 '05C85 Graph algorithms (graph-theoretic aspects) [See also 68R10, 68W05]',
 '05C90 Applications of graph theory [See also 68R10, 81Q30, 82B20, 82C20, 90C35, 92E10, 94C15]',
 '05Cxx Graph theory fFor computer science, see 68R10 g',
 '14T10 Foundations of tropical geometry and relations with algebra fFor algebraic aspects, see 15A80 g',
 '15-00 General reference works (handbooks, dictionaries, bibliographies, etc.) pertaining to linear algebra',
 '162',
 '168',
 '22E05 Local Lie groups [See also 34-XX, 35-XX,

In [ ]:
# Inspected manually because data quality right?
missing_descriptions = [
 '60B20: 60B20 Random matrices (probabilistic aspects)',
 '62H22: 62H22 Probabilistic graphical models',
 '97P30: 97P30 Systems, databases (educational aspects)',
 '05C81: 05C81 Random walks on graphs',
 '62B86: 62B86 Statistical aspects of fuzziness, sufficiency, and information',
 '68T42: 68T42 Agent technology and artificial intelligence',
 '97N80: 97N80 Mathematical software, computer programs (educational aspects)',
 '60L10: 60L10 Signatures and data streams',
 '35Q68: 35Q68 PDEs in connection with computer science',
 '68T07: 68T07 Artificial neural networks and deep learning',
 '60L20: 60L20 Rough paths',
 '62-08: 62-08 Computational methods for problems pertaining to statistics',
 '35Q79: 35Q79 PDEs in connection with classical thermodynamics and heat transfer',
 '15B48: 15B48 Positive matrices and their generalizations; cones of matrices',
 '68W50: 68W50 Evolutionary algorithms, genetic algorithms (computational aspects)',
 '62-XX: 62-XX Statistics',
 '65Zxx: 65Zxx Applications to the sciences',
 '62G86: 62G86 Nonparametric inference and fuzziness',
 '62R40: 62R40 Topological data analysis',
 '05C21: 05C21 Flows in graphs',
 "35J08: 35J08 Green's functions for elliptic equations",
 '91-10: 91-10 Mathematical modeling or simulation for problems pertaining to game theory, economics, and Finance',
 '90-05: 90-05 Experimental work for problems pertaining to operations research and mathematical programming',
 '68W27: 68W27 Online algorithms; streaming algorithms',
 '14T10: 14T10 Foundations of tropical geometry and relations with algebra',
 '62D20: 62D20 Causal inference from observational studies',
 '49Q22: 49Q22 Optimal transportation',
 '35R02: 35R02 PDEs on graphs and networks (ramified or polygonal spaces)',
 '57-08: 57-08 Computational methods for problems pertaining to manifolds and cell complexes',
 '65F55: 65F55 Numerical methods for low-rank matrix approximation; matrix compression',
 '90C17: 90C17 Robustness in mathematical programming',
 '53Z50: 53Z50 Applications of differential geometry to data and computer science',
 '35J47: 35J47 Second-order elliptic systems',
 '62R07: 62R07 Statistical aspects of big data and data science',
 '15A83: 15A83 Matrix completion problems',
 '91G80: 91G80 Financial applications of other theories',
 '54H30: 54H30 Applications of general topology to computer science (e.g., digital topology, image processing)',
 '35K08: 35K08 Heat kernel',
 '90-10: 90-10 Mathematical modeling or simulation for problems pertaining to operations research and mathematical programming',
 '78M32: 78M32 Neural and heuristic methods applied to problems in optics and electromagnetic theory',
 '62A09: 62A09 Graphical methods in statistics',
 '35Q74: 35Q74 PDEs in connection with mechanics of deformable solids',
 '55N31: 55N31 Persistent homology and applications, topological data analysis',
 '68-XX: 68-XX Computer science',
 '65D19: 65D19 Computational issues in computer and robotic vision',
 '68M25: 68M25 Computer security',
 '62-08: 62-08 Computational methods for problems pertaining to statistics',
 '92-XX: 92-XX Biology and other natural sciences',
 '35-XX: 35-XX Partial differential equations',
 '68Q87: 68Q87 Probability in computer science (algorithm analysis, random structures, phase transitions, etc.)',
 '60B20: 15B52 Random matrices (algebraic aspects) fFor probabilistic aspects, see 60B20 g',
 '91G70: 91G70 Statistical methods; risk measures [See also 62P05, 62P20]',
 '62R01: 62R01 Algebraic statistics',
 '49Q22: 35Q49 Transport equations fFor calculus of variations and optimal control, see 49Q22; for Fluid mechanics, see',
 '60B20: 60B20 Random matrices (probabilistic aspects)',
 '49M41: 49M41 PDE constrained optimization (numerical aspects)',
 '57Z25: 57Z25 Relations of manifolds and cell complexes with computer and data science',
 '49-11: 49-11 Research data for problems pertaining to calculus of variations and optimal control',
 '35Q84: 35Q84 Fokker-Planck equations',
 '42-08: 42-08 Computational methods for problems pertaining to harmonic analysis on Euclidean spaces',
 '68T09: 68T09 Computational aspects of data analysis and big data',
 '65M22: 65M22 Numerical solution of discretized equations for initial value and initial-boundary value problems involving ']

for idx, desc in enumerate(missing_descriptions):
    missing_descriptions[idx] = desc.split(':')


for idx, description in enumerate(missing_descriptions):
    m, d = missing_descriptions[idx]
    for idx2, code in enumerate(msc_cleaned['msc']):
        if msc_cleaned['msc'][idx2] == m:
            msc_cleaned['description'][idx2] = d.strip()
    

In [ ]:
# Split and add misplaced string back to ACM codes
print(f'Before: {len(ccs_cleaned)}')
acm_misplaced_string = '10010147.10010257.10010258.10010259.10010263'.split('.')
for str in acm_misplaced_string:
    ccs_cleaned.loc[-1] = str
    ccs_cleaned.index = ccs_cleaned.index + 1
    ccs_cleaned = ccs_cleaned.sort_index()
print(f'After: {len(ccs_cleaned)}')

In [ ]:
rem = msc_cleaned[msc_cleaned['description'].isnull()]['msc'].values
rem = ['60-06', '68T05', '91E40', '92B25', '92F99', '35CXX', '97R60', '68-04',
       '68T45', '68T07', '62h30', '97R40', '62-07', '68T04']
found_rem = []
for r in rem:
    desc = msc_cleaned['description'][idx]
    if r in msc_cleaned['msc'].values:
        print(f'Found: {r}')
        desc = msc_cleaned[msc_cleaned['msc'] == r]['description'].values[0]
        print(f'Description: {desc}')
        if desc is not None:
            found_rem.append(r)
    else:
        print(f'Not found: {r}')

In [ ]:
rem = set(rem) - set(found_rem)
rem = list(rem)
rem

In [ ]:
# TODO: Descriptions should have main class >> sub class >> sub sub class

In [ ]:
# Don't append before finishing TODO above
missing_descriptions.append([['97R60', '97-XX Mathematics education'], 
                            ['92B25', '92-XX Biology and other natural sciences 92Bxx Mathematical biology in general 92B25 Biological rhythms and synchronization'],
                            ['68T04', '68-XX Computer science 68Txx Artificial intelligence 68T05 Learning and adaptive systems in artificial intelligence'],
                            ['97R40', '97-XX Mathematics education'],
                            ['92F99', '92-XX Biology and other natural sciences 92Fxx Other natural sciences (mathematical treatment)'],
                            ['62H30', '62-XX Statistics 62Hxx Multivariate analysis  62H30 Classification and discrimination; cluster analysis (statistical aspects); mixture models'],
                            ['35CXX', '35Cxx Representations of solutions to partial differential equations']])

In [ ]:
missing_descriptions